In [1]:
from helper_loader import *

In [56]:
def compute_normal_euler(settings: VolumeSettings) -> np.ndarray:
    pitch = math.radians(settings.pitch)
    yaw = math.radians(settings.yaw)

    match settings.orientation:
        case Orientation.CORONAL:
            normal = [-1, 0, 0]
            rotation = Rotation.from_euler("ZY", [pitch, yaw])
        case Orientation.HORIZONTAL:
            normal = [0, 1, 0]
            rotation = Rotation.from_euler("ZX", [pitch, yaw])
        case Orientation.SAGITTAL:
            normal = [0, 0, 1]
            rotation = Rotation.from_euler("XY", [yaw, pitch])

    return rotation.apply(normal)

In [59]:
orientation = Orientation.CORONAL

match orientation:
    case Orientation.CORONAL:
        camera = coronal_camera
        normal = np.array([-1, 0, 0])
        plane_size = (228, 160)
    case Orientation.HORIZONTAL:
        camera = horizontal_camera
        normal = np.array([0, 1, 0])
        plane_size = (264, 228)
    case Orientation.SAGITTAL:
        camera = sagittal_camera
        normal = np.array([0, 0, 1])
        plane_size = (160, 264)

settings = VolumeSettings(
    orientation=orientation,
    resolution=Resolution.MICRONS_50,
    offset=0,
    pitch=0,
    yaw=0,
    shape=(264, 160, 228),
)

volume = load_volume(get_atlas_path(Resolution.MICRONS_50))
update_cameras(volume.shape)
update_axes(volume.shape)

# plane_size = (228, 160)

volume_center = np.array(
    [
        (settings.shape[0] - 1) // 2 + 1,
        (settings.shape[1] - 1) // 2,
        (settings.shape[2] - 1) // 2,
    ]
)

plotter = vedo.Plotter()
plotter += axes

match orientation:
    case Orientation.CORONAL:
        volume.tonumpy()[:, :, (volume.shape[2] - 1) // 2] = 2000
    case Orientation.HORIZONTAL:
        volume.tonumpy()[:, :, (volume.shape[2] - 1) // 2] = 2000
    case Orientation.SAGITTAL:
        volume.tonumpy()[(volume.shape[0] - 1) // 2] = 2000

volume.cmap(**get_cmap(volume, alpha=[0.005, 5]))
plotter += volume

current_normal = vedo.Arrow(
    start_pt=volume_center, end_pt=normal * 100 + volume_center, s=0.3
)
plotter += current_normal
current_plane = vedo.Plane(pos=volume_center, normal=normal, s=plane_size)
plotter += current_plane


def update_offset(widget, event) -> None:
    settings.offset = round(widget.value)
    update_plane()


def update_pitch(widget, event) -> None:
    settings.pitch = round(widget.value)
    update_plane()


def update_yaw(widget, event) -> None:
    settings.yaw = round(widget.value)
    update_plane()


def update_plane() -> None:
    global plotter, current_plane, current_normal, volume_center

    plane_center = volume_center.copy()
    plane_center[0] -= settings.offset

    plotter.remove(current_normal)
    plotter.remove(current_plane)

    normal = compute_normal_euler(settings)

    current_normal = vedo.Arrow(
        start_pt=plane_center, end_pt=normal * 100 + plane_center, s=0.3
    )
    plotter += current_normal

    current_plane = vedo.Plane(pos=plane_center, normal=normal, s=plane_size)
    plotter += current_plane


plotter.add_slider(
    update_offset, xmin=-50, xmax=132, value=0, pos="top-left", title="Offset"
)
plotter.add_slider(
    update_pitch, xmin=-90, xmax=90, value=0, pos="top-right", title="Pitch"
)
plotter.add_slider(
    update_yaw, xmin=-90, xmax=90, value=0, pos="bottom-right", title="Yaw"
)

plotter.show(camera=camera);